In [1]:
import os
import csv
import tensorflow as tf  # 2.0
import nibabel as nib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Model, Sequential
from keras.layers import Conv3D, MaxPooling3D, Dense, Dropout, Activation, Flatten 
from keras.layers import Input, concatenate
from keras import optimizers
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from augmentedvolumetricimagegenerator.generator import customImageDataGenerator
from keras.callbacks import EarlyStopping
import math
import random


# Administrative items
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Where the file is located
path = r'C:\Users\jesse\OneDrive\Desktop\Research\PD\decline'
folder = os.listdir(path)

target_size = (96, 96, 96)

Using TensorFlow backend.


In [2]:
# creating x - converting images to array
mri = []
for i in range(len(folder)):
    files = os.listdir(path + '\\' + folder[i])
    for j in range(len(files)):
        image = np.array(nib.load(path + '\\' + folder[i] + '\\' + files[j]).get_fdata())
        image = np.resize(image, target_size)
        image = np.expand_dims(image, axis=3)
        image /= 255.
        mri.append(image)

mri = np.asarray(mri)

In [3]:
mri_path = r'C:\Users\jesse\OneDrive\Desktop\Research\PD\get_skullstrip'
category = os.listdir(mri_path)

subject_id = []
for i in range(len(category)):
    ppmi_files = os.listdir(mri_path + '\\' + category[i])
    for j in range(len(ppmi_files)):
        num = ppmi_files[j][5:9]
        subject_id.append(num)

        
test_path = r'C:\Users\jesse\OneDrive\Desktop\Research\PD\Trail_Making_A_and_B.csv'

scores = []
csv_read = pd.read_csv(test_path)
patno = np.array(csv_read['PATNO'])
tmtasec = np.array(csv_read['TMTASEC'])
for i in range(len(subject_id)):
    for j in range(len(patno)):
        split = len(subject_id) - len(files)
        if subject_id[i] == str(patno[j]):
            if math.isnan(tmtasec[j]) != True:
                scores.append(tmtasec[j])
                break
            else:
                if i < split:
                    scores.append(random.randint(20., 60.))
                    break
                elif i >= split:
                    scores.append(random.randint(70., 100.))
                    break
        elif subject_id[i] not in str(patno):
            if i < split:
                scores.append(random.randint(20., 60.))
            elif i >= split:
                scores.append(random.randint(70., 100.))
            break

scores = np.asarray(scores)

In [ ]:
num = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#throw = 0
for i in range(len(scores)):
    #if scores[i] == 2. or scores[i] == 20.:
        #throw += 1
    if scores[i] < 10.:
        num[0] += 1
    elif scores[i] < 20.:
        num[1] += 1
    elif scores[i] < 30.:
        num[2] += 1
    elif scores[i] < 40.:
        num[3] += 1
    elif scores[i] < 50.:
        num[4] += 1
    elif scores[i] < 60.:
        num[5] += 1
    elif scores[i] < 70.:
        num[6] += 1
    elif scores[i] < 80.:
        num[7]+= 1
    elif scores[i] < 90.:
        num[8] += 1
    elif scores[i] < 100.:
        num[9] += 1
    else:
        num[10] += 1
print(np.percentile(num, 15), np.percentile(num, 85))
print(num)
plt.bar(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'More'], num)
plt.show

In [5]:
x = mri
y = scores
x_split, x_test, y_split, y_test = train_test_split(x, y, test_size=.2)
x_train, x_val, y_train, y_val = train_test_split(x_split, y_split, test_size=.25)
print(x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape)

(120, 96, 96, 96, 1) (40, 96, 96, 96, 1) (40, 96, 96, 96, 1) (120,) (40,) (40,)


In [4]:
batch_size = 10
num_subjects = len(mri)

model = Sequential()

model.add(Conv3D(8, [3, 3, 3], padding='same', activation='relu',
                 input_shape=[96, 96, 96, 1]))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
#model.add(Dropout(0.5))

model.add(Conv3D(16, [3, 3, 3], padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
#model.add(Dropout(0.5))

model.add(Conv3D(32, [3, 3, 3], padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))
#model.add(Dropout(0.5))

model.add(Conv3D(64, [3, 3, 3], padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model.add(Conv3D(128, [3, 3, 3], padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

opt = optimizers.Adam(lr=1e-4)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

Epoch 1/15
16/16 [==============================] - 156s 10s/step - loss: 33.1357 - accuracy: 0.2500
Epoch 2/15
16/16 [==============================] - 147s 9s/step - loss: 33.1358 - accuracy: 0.2500
Epoch 3/15
16/16 [==============================] - 153s 10s/step - loss: 33.1358 - accuracy: 0.2500
Epoch 4/15
16/16 [==============================] - 153s 10s/step - loss: 33.1359 - accuracy: 0.2500
Epoch 5/15
16/16 [==============================] - 151s 9s/step - loss: 33.1358 - accuracy: 0.2500
Epoch 6/15
16/16 [==============================] - 151s 9s/step - loss: 33.1358 - accuracy: 0.2500
Epoch 7/15
16/16 [==============================] - 151s 9s/step - loss: 33.1358 - accuracy: 0.2500
Epoch 8/15


KeyboardInterrupt: 

In [ ]:
train_datagen = customImageDataGenerator(horizontal_flip=True)

val_datagen = customImageDataGenerator()

test_datagen = customImageDataGenerator()



training_set = train_datagen.flow(x_train, y_train, batch_size=batch_size)

validation_set = val_datagen.flow(x_val, y_val, batch_size=batch_size)

testing_set = test_datagen.flow(x_test, y_test, batch_size=batch_size, shuffle=False)



callbacks = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit_generator(
                    training_set,
                    steps_per_epoch = len(x_train)//batch_size,
                    epochs = 20,
                    callbacks = [callbacks],
                    validation_data = validation_set,
                    validation_steps = len(x_val)//batch_size
                    )
score = model.predict_generator(testing_set, steps=len(x_test)//batch_size)

In [ ]:
def accuracy_score(y_pred, y_test):
    count = 0
    for i in range(len(y_test)):
        error = 5.
        if (y_pred[i] < y_test[i] + error) and (y_pred[i] > y_test[i] - error):
            count += 1
    accuracy = count/len(y_test)
    return accuracy


y_pred = model.predict(x_test, batch_size=batch_size)

for i in range(len(y_pred)):
    print('Predicted score:', y_pred[i], 'True score:', y_test[i])

accuracy_score(y_pred, y_test)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(1)
plt.plot(loss)
plt.plot(val_loss)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.title('Loss')